# Axegine Optimal B/A Pricing Model

AJ Lindeman

June 2018

## Problem (re) statement and analysis

We are making markets in a single risky asset having a lognormal price process
$$
dP_t = P_t\left( \mu\,dt + \sigma\,dW_t\right).
$$

Buy and sell orders are modeled as a Poisson counting process with densities $\lambda_0^b$ and $\lambda_0^s$. Our quotes take the form
$$
P_te^s_t
$$
and the probability the quote is accepted is modeled as
$$
\lambda_1^\cdot(s_t) = e^{-\beta s^\cdot_t}.
$$
Note $\beta$ is a parameter reflecting the fact that we cannot guarantee winning the trade by setting the spread to zero:
$$
P(\text{agent}\,\text{buys}\,\vert\, s\in[0,\delta)) = \int_0^\delta e^{-\beta s}\,ds=1-\frac 1\beta e^{-\beta\delta} \approx 1 - \frac 1\beta -\delta.
$$
Obviously we must have $\beta\geq 1$, and really only $\beta > 1$ is realistic, as other dealers may be better buyers and sellers (perhaps using a different utility function than us).

_**Claim**: Accepted trades arrive at rate $\lambda^\cdot = \lambda_0^\cdot \times \lambda_1^\cdot.$_ (TODO)

Trade quantities are denoted by $Q$. In this setup the change in inventory is goverened by
$$
dI_t = \left(Q^s\lambda^s - Q^b\lambda^b\right)\,dt
$$
In words, the difference between expected agent sells (our buys) and expected agent buys (our sells) over a small time period.

The change in our cash account is similar, except that we have to reflect the transaction prices:
$$
dC_t = P_t\left(Q^b\lambda^b e^{s^A} - Q^s\lambda^s e^{-s^B} \right)\,dt
$$
As a mental check, if we take $Q=1$ and $s=s^A=s^B$ and $\lambda = \lambda^b = \lambda^a$ we have
$$
dC_t = Q\lambda P_t\left(e^s - e^{-s}\right)\,dt\approx 2 Q\lambda P_t s\,dt
$$
which is twice the ("proportional") spread times the expected quantity traded. (I.e. the spread convention here is a percentage of value, not the more usual absolute amount.)

Our wealth process is simply the sum of the cash account and value of our position (taken at mid):
$$
\Pi_t = C_t + I_t P_t,
$$
and we are asked to choose a policy $\left\{s^A_t, s^B_t\right\}$ that maximizes exponential utility at some terminal time $T$:
$$
   \max {\bf E}\left[-e^{-\Pi(T)}\right]
$$

### Observations

* No time value of money (risk-free rate is zero).
* Can borrow indefinitely, implying no upper limit on long position
* Can short indefinitely
* No penalty for holding inventory; no link between spreads and inventory
* No explicity hedging of position, although $P_t$ could reprsent the price dynamics of a hedged portfolio
* Might re-parameterize to the arrival rate of unit orders - i.e. collapse $Q$ and $\lambda$.

### Overall solution strategy

* Simulate to check understanding and validate expected sensitivites, such as 
  * Don't really see any time-dependence here
  * Similarly the policy won't depend on position, 
  * So perhaps it doesn't depend on wealth, and it's a constant 
  * Bid and ask spreads won't be symmetric - the asymmetry should be controlled by $\mu$ and the relative arrival rates of buy and sell orders (and relative buy and sell sizes).
  * Bid and ask spreads increase with $\sigma$
* Write down the SDE for the wealth process
* Find the infinitessimal generator and pose the optimization as an HJB equation
* Assume a classical solution and "differentiate" to find the optimal policy
* Analyze the PDE more to see what we can say about expected terinal wealth

## Simulation

In [1]:
import numpy as np
import matplotlib.pylab as plt

In [2]:
T = 250 # days
dt = 0.125 # 8 trading hours per day
Qs = 1.0 # unit customer sell size
Qb = 1.0 # unit customer buy size
mu = 0.02/250.0  # 2% per annum
sig = 0.01/np.sqrt(T) # 1% asset vol (eg high-quality bond)
lam0s = 0.6033*8.0/T # 33% chance per dt via P(N(1/8)=1) = lam/8 exp(-lam/8) = 0.33
lam0b = lam0s # let's make buys and sells symmetric
beta = 2.0 # so we are only 50% likely to win a trade with a zero spread

In [3]:
ns = 20000
P0 = 100.0
Pt = np.zeros((T+1, ns))
Ct = np.zeros((T+1, ns))
It = np.zeros((T+1, ns))
Vt = np.zeros((T+1, ns))
Pt[0,:] = P0
sA = 0.02
sB = 0.005
for t in range(T):
    # price process
    Pt[t+1,:] = Pt[t,:] * np.exp((mu-sig**2)*dt) * np.exp(sig*np.random.normal(scale=np.sqrt(dt), size=ns))
    # order arrivals
    b_t = Qb*np.random.poisson(16*lam0b, ns) # something is not right with lam0s...
    s_t = Qs*np.random.poisson(16*lam0s, ns)
    # trades need to be won 
    b_t[np.random.uniform(size=ns) < 1.0-1.0/beta] = 0.0 # implemented as lost
    s_t[np.random.uniform(size=ns) < 1.0-1.0/beta] = 0.0
    It[t+1,:] = s_t - b_t + It[t,:]
    Ct[t+1,:] = Ct[t,:] + Pt[t+1,:]*(b_t*np.exp(sA)-s_t*np.exp(-sB))
    Vt[t+1,:] = Ct[t+1,:] + It[t+1,:]*Pt[t+1,:]*(b_t*np.exp(sA)-s_t*np.exp(-sB))
    
print(np.mean(Vt), np.std(Vt))

16.8932734493 785.322886866
